<a href="https://colab.research.google.com/github/Surya-dev-lang/datascience/blob/master/ResturantSentimentAnlaysis_NLP_RegularWay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk

In [0]:
dataset = pd.read_csv("Restaurant_Reviews.tsv", delimiter='\t')

In [4]:
dataset.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [8]:
dataset.groupby('Liked').describe()

Review                                               
       count unique                                top freq
Liked                                                      
0        500    497  I would not recommend this place.    2
1        500    499                 I love this place.    2

In [0]:
Y = dataset['Liked']

In [12]:
#Tokenization
nltk.download('all')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [0]:
def split_tokens(review):
  review=review.lower()
  word_tokens = word_tokenize(review)
  return word_tokens

In [0]:
dataset['tokenized_review']= dataset.apply(lambda row: split_tokens(row['Review']), axis=1)

In [16]:
dataset.head()

,Review,Liked,tokenized_review
0,Wow... Loved this place.,1,"[wow, ..., loved, this, place, .]"
1,Crust is not good.,0,"[crust, is, not, good, .]"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nas..."
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an..."


In [0]:
# LEMMATIZATION
from nltk.stem.wordnet import WordNetLemmatizer

def split_into_lemmas(review):
  lemmatizer = WordNetLemmatizer()
  lemma = []
  for word in review:
    a = lemmatizer.lemmatize(word)
    lemma.append(a)
  return lemma

In [0]:
dataset['lemmatized_review'] = dataset.apply(lambda row: split_into_lemmas(row['tokenized_review']), axis=1)

In [22]:
dataset.head()

,Review,Liked,tokenized_review,lemmatized_review
0,Wow... Loved this place.,1,"[wow, ..., loved, this, place, .]","[wow, ..., loved, this, place, .]"
1,Crust is not good.,0,"[crust, is, not, good, .]","[crust, is, not, good, .]"
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nas...","[not, tasty, and, the, texture, wa, just, nast..."
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho...","[stopped, by, during, the, late, may, bank, ho..."
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an...","[the, selection, on, the, menu, wa, great, and..."


In [0]:
# Stop Word Removal

from nltk.corpus import stopwords

def stopword_removal(review):
  stop_words = set(stopwords.words('english'))
  filtered_sentence = []
  filtered_sentence = ' '.join([word for word in review if word not in stop_words])
  return filtered_sentence

In [0]:
dataset['preprocessed_review'] = dataset.apply(lambda row: stopword_removal(row['lemmatized_review']), axis=1)

In [25]:
dataset.head()

,Review,Liked,tokenized_review,lemmatized_review,preprocessed_review
0,Wow... Loved this place.,1,"[wow, ..., loved, this, place, .]","[wow, ..., loved, this, place, .]",wow ... loved place .
1,Crust is not good.,0,"[crust, is, not, good, .]","[crust, is, not, good, .]",crust good .
2,Not tasty and the texture was just nasty.,0,"[not, tasty, and, the, texture, was, just, nas...","[not, tasty, and, the, texture, wa, just, nast...",tasty texture wa nasty .
3,Stopped by during the late May bank holiday of...,1,"[stopped, by, during, the, late, may, bank, ho...","[stopped, by, during, the, late, may, bank, ho...",stopped late may bank holiday rick steve recom...
4,The selection on the menu was great and so wer...,1,"[the, selection, on, the, menu, was, great, an...","[the, selection, on, the, menu, wa, great, and...",selection menu wa great price .


In [0]:
X = dataset['preprocessed_review']

In [0]:
# Term Document Matrix
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(X).toarray()
Y = dataset['Liked'].values

In [32]:
X.shape
Y.shape

(1000,)

In [0]:
from sklearn.model_selection import train_test_split  

X_train,X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.1)

In [48]:
from sklearn.ensemble import RandomForestClassifier

rfclassifier = RandomForestClassifier(max_depth=5, n_estimators=15, max_features=60,random_state=10)

rfclassifier = rfclassifier.fit(X_train, Y_train)

rfscore=rfclassifier.score(X_test, Y_test)

print('Random Forest classification after model tuning',rfscore)

Random Forest classification after model tuning 0.7


In [0]:
# With TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(max_features=1500)
X = tv.fit_transform(X).toarray()


In [50]:
from sklearn.svm import SVC
sv = SVC(kernel="linear", C=15,random_state=10)
sv = sv.fit(X_train, Y_train)
score = sv.score(X_test, Y_test)
score

0.75

In [57]:
import xgboost as xgb
xb = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xb.fit(X_train, Y_train)
score = sv.score(X_test, Y_test)
score

0.75